In [85]:
# Titanic with Pipeline

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Path of the file to read
Titanic_train_path = 'train.csv'
Titanic_test_path = 'test.csv'

train_data = pd.read_csv(Titanic_train_path)
test_data = pd.read_csv(Titanic_test_path)

y = train_data.Survived

# Create X
features = ['Pclass','Name','Sex','Age','SibSp','Parch','Fare']
X = train_data[features]
X_test=test_data[features]

# Title extraction

X_title= (train_data.Name.str.split(", ", n = 1, expand = True)) 
X_title.loc[:] = X_title[1].str.split(".", n = 1, expand = True) 
X.loc[:,"Name"]=X_title[0]
X.rename(columns = {"Name": "Title"},inplace = True) 

X_title_test= (test_data.Name.str.split(", ", n = 1, expand = True)) 
X_title_test.loc[:] = X_title_test[1].str.split(".", n = 1, expand = True) 
X_test.loc[:,"Name"]=X_title_test[0]
X_test.rename(columns = {"Name": "Title"},inplace = True) 


# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

#Preprocessor
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in train_X.columns if
                    train_X[cname].nunique() < 10 and 
                    train_X[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in train_X.columns if 
                train_X[cname].dtype in ['int64', 'float64']]


# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Model and pipeline

Titanic_model = RandomForestRegressor(n_estimators=100,random_state=1)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', Titanic_model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(train_X, train_y)

# Make validation predictions and calculate mean absolute error
val_predictions_train = clf.predict(val_X).round()
accuracy_train = 1-sum(abs(val_predictions_train-val_y))/len(val_y)

print("Le pourcentage de prédictions justes est:",round(100*accuracy_train,1),"%")

# Test prediction

clf.fit(X, y)

# Make validation predictions and calculate mean absolute error
val_predictions_test = clf.predict(X_test)
val_predictions_test=np.int8(val_predictions_test+0.5)

# Save test predictions to file
output = pd.DataFrame({'PassengerId': test_data['PassengerId'],
                       'Survived': val_predictions_test})
output.to_csv('submission_Titanic.csv', index=False)

C:\Users\Johann\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Johann\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Le pourcentage de prédictions justes est: 78.9 %
